In [1]:
from binance.client import Client
from binance.enums import *
from binance import ThreadedWebsocketManager
from binance.exceptions import BinanceAPIException, BinanceOrderException
from time import sleep

In [2]:
api_key = '你的測試api_key'
api_secret = '你的測試api_secret'

client = Client(api_key, api_secret)

client.API_URL = 'https://testnet.binance.vision/api'

In [4]:
balance = client.get_asset_balance(asset='USDT')
print(balance)

{'asset': 'USDT', 'free': '2777.26647600', 'locked': '0.00000000'}


In [10]:
try:
    buy_limit = client.create_order(
        symbol='ETHUSDT',
        side=SIDE_SELL ,
        type=ORDER_TYPE_LIMIT,
        timeInForce=TIME_IN_FORCE_GTC,
        quantity=1,
        price=3000)
    print(buy_limit)
except BinanceAPIException as e:
    # error handling goes here
    print(e)
except BinanceOrderException as e:
    # error handling goes here
    print(e)

{'symbol': 'ETHUSDT', 'orderId': 25222, 'orderListId': -1, 'clientOrderId': 'qRt76af4d220L9HEDBXcSY', 'transactTime': 1629855021149, 'price': '3000.00000000', 'origQty': '1.00000000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT', 'side': 'SELL', 'fills': []}


In [6]:
from binance.enums import *
order = client.create_order(
    symbol='ETHUSDT',
    side=SIDE_BUY,
    type=ORDER_TYPE_LIMIT,
    timeInForce=TIME_IN_FORCE_GTC,
    quantity=1,
    price=3338)

BinanceAPIException: APIError(code=-2010): Account has insufficient balance for requested action.

In [7]:
price = {'BTCUSDT': None, 'error': False}

In [11]:
def btc_pairs_trade(msg):
    ''' define how to process incoming WebSocket messages '''
    if msg['e'] != 'error':
        price['BTCUSDT'] = float(msg['c'])
    else:
        price['error'] = True

In [12]:
bsm = ThreadedWebsocketManager()
bsm.start()
bsm.start_symbol_ticker_socket(symbol='BTCUSDT', callback=btc_pairs_trade)

'btcusdt@ticker'

In [13]:
while not price['BTCUSDT']:
    # wait for WebSocket to start streaming data
    sleep(0.1)

In [14]:
while True:
    # error check to make sure WebSocket is working
    if price['error']:
        # stop and restart socket
        bsm.stop()
        sleep(2)
        bsm.start()
        price['error'] = False

    else:
        if price['BTCUSDT'] > 35000:
            try:
                order = client.order_market_buy(symbol='ETHUSDT', quantity=1)
                break
            except Exception as e:
                print(e)

    sleep(1)

In [15]:
bsm.stop()

In [16]:
orders = client.get_all_orders(symbol='ETHUSDT')
print(orders)

[{'symbol': 'ETHUSDT', 'orderId': 22874, 'orderListId': -1, 'clientOrderId': 'npKYgxvJnQOIheH2fRJdek', 'price': '3338.00000000', 'origQty': '1.00000000', 'executedQty': '1.00000000', 'cummulativeQuoteQty': '989.00000000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'LIMIT', 'side': 'BUY', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': 1629691022123, 'updateTime': 1629691022123, 'isWorking': True, 'origQuoteOrderQty': '0.00000000'}, {'symbol': 'ETHUSDT', 'orderId': 22877, 'orderListId': -1, 'clientOrderId': 'BgKNVqFblUt0uxw0Xmle32', 'price': '3338.00000000', 'origQty': '1.00000000', 'executedQty': '1.00000000', 'cummulativeQuoteQty': '989.00000000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'LIMIT', 'side': 'BUY', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': 1629691088079, 'updateTime': 1629691088079, 'isWorking': True, 'origQuoteOrderQty': '0.00000000'}, {'symbol': 'ETHUSDT', 'orderId': 22878, 'orderListId': -1, 'clientOrderId': 'fESMBU1h